In [7]:
from __future__ import print_function

import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import torchvision.datasets as dset
import torchvision.transforms as transforms
import io
import matplotlib.pyplot as plt
import numpy as np
import pickle
from matplotlib.pyplot import figure

# Set random seed for reproducibility
manualSeed = 42

# print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)


# Number of workers for dataloader
workers = 1

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 100

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1


def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array(tokens[1:]).astype(float)
    return data


# fast_text_vectors = load_vectors('coco/wiki-news-300d-1M.vec')

In [8]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)



class ReverseDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, embeddings):
        self.dataset = dataset
        self.embeddings = embeddings

    def __len__(self):
        return len(self.dataset) * 5

    def __getitem__(self, i):
        img, target = self.dataset[int(i / 5)]

        encoding = self.embeddings[target[i % 5]]
        return img, encoding


class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz+384, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)


class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            #nn.Conv2d(ndf * 8, (ndf*16), 4, 1, 0, bias=False),
            #nn.Flatten(),
            #nn.Linear(1024, 384),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)


In [ ]:
figure(figsize=(8, 8), dpi=100)

dataset = dset.CocoCaptions(
    root='coco/train2014',
    annFile='coco/annotations/captions_train2014.json',
    transform=transforms.Compose([
                           transforms.Resize(image_size),
                           transforms.CenterCrop(image_size),
                           transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                       ])
)

pickle_file = open('sbert-captions.pkl', 'rb')     
db = pickle.load(pickle_file)

dataset = ReverseDataset(dataset, db)

print('Number of samples: ', len(dataset))
img, target = dataset[3]

# Create the dataloader
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=workers
)

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")


# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.02.
netG.apply(weights_init)

# Print the model
print(netG)

# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
print(netD)

# Initialize BCELoss function
criterion = nn.BCELoss()
cos = nn.CosineSimilarity(dim=1, eps=1e-6)


# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

target_input = torch.unsqueeze(torch.unsqueeze(iter(dataloader).next()[1][:64], -1), -1)
target_input = target_input.type(torch.float).to(device)

fixed_embedding = target_input

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))


real_label = 1.
fake_label = 0.


# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        
        # plt.imshow(np.moveaxis(vutils.make_grid(real_cpu[:64], padding=2, normalize=True).detach().cpu().numpy(), 0, -1), interpolation='nearest')
        # plt.savefig('{}_{}_real.png'.format(i, epoch))
        
        
        target = data[1].to(device)
        b_size = real_cpu.size(0)
        
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        
        
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        # Train with all-fake batch
        # Generate batch of latent vectors
        
        noise_vector = torch.randn(b_size, nz, 1, 1, device=device, dtype=torch.float)
        target_input = torch.unsqueeze(torch.unsqueeze(target, -1), -1)
        target_input = target_input.type(torch.float)
        
        noise = torch.cat((torch.randn(b_size, nz, 1, 1, device=device), target_input), 1)
        # Generate fake image batch with G
        fake = netG(noise)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        
        label.fill_(fake_label)
        
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        
        label.fill_(real_label)
        
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (i % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            
            plt.imshow(np.moveaxis(real_cpu[0].detach().cpu().numpy(), 0, -1), interpolation='nearest')
            plt.savefig('images/real_{}_{}.png'.format(i, epoch))
            
            with torch.no_grad():
                fake = netG(torch.cat((fixed_noise, fixed_embedding), 1)).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
            plt.imshow(np.moveaxis(vutils.make_grid(fake, padding=2, normalize=True).detach().cpu().numpy(), 0, -1), interpolation='nearest')
            plt.savefig('images/{}_{}.png'.format(i, epoch))
            
        torch.save(netG, "model_files/netG_{}.model".format(epoch))
        torch.save(netD, "model_files/netD_{}.model".format(epoch))

        iters += 1

loading annotations into memory...
Done (t=0.68s)
creating index...
index created!
Number of samples:  413915
Generator(
  (main): Sequential(
    (0): ConvTranspose2d(484, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12):

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


[0/100][0/3234]	Loss_D: 1.7735	Loss_G: 6.1188	D(x): 0.5267	D(G(z)): 0.5597 / 0.0038
[0/100][50/3234]	Loss_D: 0.3219	Loss_G: 20.9967	D(x): 0.8271	D(G(z)): 0.0000 / 0.0000
[0/100][100/3234]	Loss_D: 0.5574	Loss_G: 6.2981	D(x): 0.8111	D(G(z)): 0.2210 / 0.0034
[0/100][150/3234]	Loss_D: 0.7598	Loss_G: 3.9304	D(x): 0.7601	D(G(z)): 0.2703 / 0.0375
[0/100][200/3234]	Loss_D: 0.3506	Loss_G: 5.2352	D(x): 0.8760	D(G(z)): 0.1225 / 0.0077
[0/100][250/3234]	Loss_D: 0.0690	Loss_G: 8.1154	D(x): 0.9532	D(G(z)): 0.0077 / 0.0004
[0/100][300/3234]	Loss_D: 0.2761	Loss_G: 3.2240	D(x): 0.8432	D(G(z)): 0.0678 / 0.0696
[0/100][350/3234]	Loss_D: 1.1287	Loss_G: 4.3356	D(x): 0.7224	D(G(z)): 0.4397 / 0.0192
[0/100][400/3234]	Loss_D: 0.4068	Loss_G: 3.2455	D(x): 0.8108	D(G(z)): 0.1274 / 0.0588
[0/100][450/3234]	Loss_D: 0.8248	Loss_G: 6.9595	D(x): 0.5787	D(G(z)): 0.0026 / 0.0019


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


[0/100][500/3234]	Loss_D: 0.8023	Loss_G: 5.1981	D(x): 0.7517	D(G(z)): 0.3232 / 0.0091
[0/100][550/3234]	Loss_D: 0.1942	Loss_G: 4.3123	D(x): 0.9004	D(G(z)): 0.0713 / 0.0177
[0/100][600/3234]	Loss_D: 0.2075	Loss_G: 4.6666	D(x): 0.9158	D(G(z)): 0.0988 / 0.0127
[0/100][650/3234]	Loss_D: 0.3560	Loss_G: 6.1429	D(x): 0.8212	D(G(z)): 0.0576 / 0.0063
[0/100][700/3234]	Loss_D: 2.7415	Loss_G: 1.0669	D(x): 0.2420	D(G(z)): 0.0230 / 0.5648
[0/100][750/3234]	Loss_D: 0.2185	Loss_G: 5.0168	D(x): 0.8853	D(G(z)): 0.0762 / 0.0097
[0/100][800/3234]	Loss_D: 0.3225	Loss_G: 5.5874	D(x): 0.9263	D(G(z)): 0.1996 / 0.0061
[0/100][850/3234]	Loss_D: 0.1192	Loss_G: 4.3095	D(x): 0.9299	D(G(z)): 0.0351 / 0.0182
[0/100][900/3234]	Loss_D: 1.6431	Loss_G: 6.6781	D(x): 0.3549	D(G(z)): 0.0005 / 0.0021
[0/100][950/3234]	Loss_D: 0.2749	Loss_G: 4.0682	D(x): 0.8951	D(G(z)): 0.1315 / 0.0268


...

[3/100][1500/3234]	Loss_D: 0.3566	Loss_G: 2.7673	D(x): 0.7917	D(G(z)): 0.0928 / 0.0822
[3/100][1550/3234]	Loss_D: 0.5117	Loss_G: 2.1538	D(x): 0.8557	D(G(z)): 0.2464 / 0.1650
[3/100][1600/3234]	Loss_D: 0.3451	Loss_G: 2.6085	D(x): 0.8346	D(G(z)): 0.1314 / 0.0993
[3/100][1650/3234]	Loss_D: 0.3594	Loss_G: 1.3930	D(x): 0.7717	D(G(z)): 0.0774 / 0.2920
[3/100][1700/3234]	Loss_D: 0.5541	Loss_G: 4.2870	D(x): 0.9477	D(G(z)): 0.3597 / 0.0183
[3/100][1750/3234]	Loss_D: 0.4282	Loss_G: 3.4067	D(x): 0.9057	D(G(z)): 0.2515 / 0.0444
[3/100][1800/3234]	Loss_D: 0.2515	Loss_G: 2.6459	D(x): 0.8551	D(G(z)): 0.0783 / 0.0939
[3/100][1850/3234]	Loss_D: 1.0260	Loss_G: 1.5579	D(x): 0.4323	D(G(z)): 0.0237 / 0.2599
[3/100][1900/3234]	Loss_D: 0.3207	Loss_G: 3.2489	D(x): 0.9106	D(G(z)): 0.1845 / 0.0559
[3/100][1950/3234]	Loss_D: 0.3918	Loss_G: 2.3951	D(x): 0.7306	D(G(z)): 0.0475 / 0.1192


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


[3/100][2000/3234]	Loss_D: 0.7805	Loss_G: 1.6525	D(x): 0.5983	D(G(z)): 0.1118 / 0.2466
[3/100][2050/3234]	Loss_D: 0.3826	Loss_G: 3.8456	D(x): 0.9433	D(G(z)): 0.2450 / 0.0313
[3/100][2100/3234]	Loss_D: 0.3795	Loss_G: 2.2247	D(x): 0.7576	D(G(z)): 0.0714 / 0.1584
[3/100][2150/3234]	Loss_D: 0.2580	Loss_G: 3.2711	D(x): 0.8222	D(G(z)): 0.0448 / 0.0547
[3/100][2200/3234]	Loss_D: 0.3394	Loss_G: 2.4701	D(x): 0.8092	D(G(z)): 0.1016 / 0.1110
[3/100][2250/3234]	Loss_D: 0.5564	Loss_G: 2.5092	D(x): 0.8262	D(G(z)): 0.2577 / 0.1071
[3/100][2300/3234]	Loss_D: 0.4965	Loss_G: 3.6286	D(x): 0.9077	D(G(z)): 0.2930 / 0.0383
[3/100][2350/3234]	Loss_D: 0.2391	Loss_G: 3.7852	D(x): 0.9272	D(G(z)): 0.1356 / 0.0315
[3/100][2400/3234]	Loss_D: 0.4014	Loss_G: 2.5587	D(x): 0.8074	D(G(z)): 0.1523 / 0.1002
[3/100][2450/3234]	Loss_D: 0.4197	Loss_G: 2.0424	D(x): 0.7416	D(G(z)): 0.0819 / 0.1714


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


[3/100][2500/3234]	Loss_D: 0.5869	Loss_G: 1.4889	D(x): 0.6384	D(G(z)): 0.0688 / 0.2876
[3/100][2550/3234]	Loss_D: 0.4817	Loss_G: 3.7071	D(x): 0.8589	D(G(z)): 0.2479 / 0.0357
[3/100][2600/3234]	Loss_D: 0.6849	Loss_G: 0.7600	D(x): 0.5606	D(G(z)): 0.0423 / 0.5163
[3/100][2650/3234]	Loss_D: 0.4135	Loss_G: 3.4637	D(x): 0.8672	D(G(z)): 0.2149 / 0.0423
[3/100][2700/3234]	Loss_D: 0.5519	Loss_G: 2.2802	D(x): 0.6211	D(G(z)): 0.0157 / 0.1447
[3/100][2750/3234]	Loss_D: 0.4703	Loss_G: 3.2073	D(x): 0.9179	D(G(z)): 0.2821 / 0.0575
[3/100][2800/3234]	Loss_D: 0.7133	Loss_G: 2.6412	D(x): 0.5478	D(G(z)): 0.0130 / 0.1067
[3/100][2850/3234]	Loss_D: 0.9651	Loss_G: 6.1801	D(x): 0.9887	D(G(z)): 0.5571 / 0.0034
